# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Using-the-CPSC-API-(unsuccessfully)" data-toc-modified-id="Using-the-CPSC-API-(unsuccessfully)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Using the CPSC API (unsuccessfully)</a></div><div class="lev1 toc-item"><a href="#Cleaning-up-Raw-Data" data-toc-modified-id="Cleaning-up-Raw-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Cleaning up Raw Data</a></div><div class="lev2 toc-item"><a href="#API-Data" data-toc-modified-id="API-Data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>API Data</a></div><div class="lev2 toc-item"><a href="#NEISS-Data" data-toc-modified-id="NEISS-Data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>NEISS Data</a></div><div class="lev1 toc-item"><a href="#Beginning-of-Analysis" data-toc-modified-id="Beginning-of-Analysis-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Beginning of Analysis</a></div><div class="lev2 toc-item"><a href="#API-Data" data-toc-modified-id="API-Data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>API Data</a></div><div class="lev3 toc-item"><a href="#Natural-Language-Processing" data-toc-modified-id="Natural-Language-Processing-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Natural Language Processing</a></div><div class="lev1 toc-item"><a href="#Appendix" data-toc-modified-id="Appendix-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Appendix</a></div><div class="lev2 toc-item"><a href="#Crosstab-of-Items-and-Injury" data-toc-modified-id="Crosstab-of-Items-and-Injury-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Crosstab of Items and Injury</a></div>

In [ ]:
import os
import pickle
import re

from IPython.display import display
import numpy as np
import pandas as pd
import unirest
import nltk
from gensim.models import word2vec

# Using the CPSC API (unsuccessfully)

Here I try to make a request from the CPSC API and it returns a blank even though the status code is 200.

In [ ]:
key = os.environ.get('CPSC_KEY')
resp = unirest.get('http://www.saferproducts.gov/webapi/Cpsc.Cpsrms.Web.Api.svc/',
                   auth=(key,''), headers={"Accept": "application/json"})

# Cleaning up Raw Data

## API Data
Rather than calling from the API, we received the data from the epidemiologist from the CPSC in a raw `.txt` format. However it is a JSON file and there are levels of nesting that we would need to parse out so I created this script below.

In [ ]:
data = pd.read_json('data/raw_api_data.txt')

cols_to_parse = ['Gender', 'SeverityType', 'Locale', 'ProductCategory']
cols_to_add = [['GenderDescription','GenderId','GenderPublicName'],
['IncidentDetails','SeverityTypeDescription','SeverityTypePublicName'],
['LocaleDescription','LocalePublicName'],
['ProductCategoryDescription','ProductCategoryPublicName']]
new_df = pd.DataFrame()
cols_dict = dict(zip(cols_to_parse, cols_to_add))

for key in cols_dict:
    placeholder = pd.DataFrame(columns = cols_dict[key])
    for (i, row) in data.iterrows():
        e = row[key]
        value_holder = []
        for item in cols_dict[key]:
            try:
                component = e[item]
            except:
                component = 'Missing'
            value_holder.append(component)
        placeholder.loc[i, :] = value_holder
    if new_df.shape[0] == 0:
        new_df = placeholder
    else:
        new_df = pd.concat([new_df, placeholder], axis=1)
        
new_df2 = pd.concat([data, new_df], axis=1)
new_df2 = new_df2.drop(['CompanyComments', 'Gender','IncidentDocuments','IncidentDetails', 'Locale', 'ProductCategory',
                      'RelationshipType','SeverityType', 'SourceType'], axis=1)
new_df2.to_pickle('data/cleaned_api_data')

## NEISS Data

Combining all the NEISS data from 2009. 2015 was different formatted so I changed it manually and combined all the files into a single data frame. The raw NEISS data wasn't so bad to work with but I figure we could compress it just to save space in the `processed` folder.

In [9]:
#compressing raw data from 2009 to 2014 and saving it in the processed folder
raw_path = os.environ.get('RAW_FILE_PATH')
cleaned_path = os.environ.get('PROCESSED_PATH')
for excel in os.listdir(raw_path):
    data = pd.read_csv(excel)
    data.to_csv(os.path.join(cleaned_path, excel), compression='gzip')

In [62]:
# combining it to a single dataframe
data = pd.DataFrame()
for i in range(2009, 2015):
    filepath = '~/cpsc/data/processed/neiss/neiss-' + str(i) + '.csv'
    temp = pd.read_csv(filepath, compression='gzip')
    temp['year'] = i
    data = pd.concat([data, temp])
data['narr2'] = np.nan

In [78]:
# processing the 2015 manually
latest = pd.read_csv('~/cpsc/data/processed/neiss/neiss-2015.csv', compression='gzip')
latest['year'] = '2015'
columns = latest.columns.values.tolist()
new_cols = columns[:-2]
new_cols.append(columns[-1])
new_cols.append(columns[-2])
final = pd.concat([data, latest.ix[:,new_cols]])
final.to_csv('~/cpsc/data/processed/neiss-combined.csv', compression='gzip')
final.head()

,CPSC Case #,Unnamed: 0,age,body_part,diag,diag_other,disposition,fmv,location,narr1,...,prod1,prod2,psu,race,race_other,sex,stratum,trmt_date,weight,year
0,90101432,0,5,89,64,NaN,1,0,1,NaN,...,1807,NaN,61,Other / Mixed Race,HISPANIC,Male,V,01-01-09,15.3491,2009
1,90101434,1,51,77,53,NaN,1,0,1,NaN,...,899,NaN,61,White,NaN,Male,V,01-01-09,15.3491,2009
2,90101435,2,2,76,59,NaN,1,0,1,NaN,...,4057,NaN,61,White,NaN,Female,V,01-01-09,15.3491,2009
3,90101436,3,20,93,53,NaN,1,0,1,NaN,...,1884,NaN,61,White,NaN,Male,V,01-01-09,15.3491,2009
4,90101437,4,20,34,57,NaN,1,0,9,NaN,...,3283,NaN,61,White,NaN,Male,V,01-01-09,15.3491,2009


In [79]:
# quick check
final.year.value_counts()

2010    405710
2011    396502
2012    394383
2009    391944
2013    376926
2014    367492
2015    359129
Name: year, dtype: int64

# Beginning of Analysis

Just answering some of the questions the CPSC had on their data. I start with the hackpad. Here I open the cleaned api data using pickle since I saved it in a pickle format (saving it as a csv ran into encoding error and I didn't want to corrupt the data further)

## API Data

In [4]:
# neiss = pd.read_csv('/NEISS-data-2015-updated-APRIL2016.csv')
data = pickle.load(open('/Users/cipherpol/cpsc/data/processed/cleaned_api_data', 'rb'))
# neiss = pd.read_csv('~/cpsc/data/processed/neiss-combined.csv', compression='gzip')

In [104]:
#Looking at the item that is reported the most
data.ProductCategoryPublicName.value_counts()[0:9]

Electric Ranges or Ovens (Excl Counter-top Ovens)    2704
Dishwashers                                          1605
Microwave Ovens                                      1095
Footwear                                              949
Refrigerators                                         888
Gas Ranges or Ovens                                   872
Computers (Equipment and Electronic Games)            838
Electric Coffee Makers or Teapots                     748
Nonmetal Cookware (Nonelectric)                       530
Name: ProductCategoryPublicName, dtype: int64

In [98]:
# Most producted reported by men
data.ix[data['GenderPublicName'] == 'Male', :].ProductCategoryPublicName.value_counts()[0:9]

Electric Ranges or Ovens (Excl Counter-top Ovens)    582
Dishwashers                                          411
Microwave Ovens                                      276
Computers (Equipment and Electronic Games)           272
Refrigerators                                        246
Electric Coffee Makers or Teapots                    220
Gas Ranges or Ovens                                  214
Cribs                                                199
Gas or Lp Grills or Stoves (For Outdoor Use)         181
Name: ProductCategoryPublicName, dtype: int64

In [105]:
# Most producted reported by men
data.ix[data['GenderPublicName'] == 'Female', :].ProductCategoryPublicName.value_counts()[0:9]

Electric Ranges or Ovens (Excl Counter-top Ovens)    1075
Footwear                                              782
Dishwashers                                           475
Computers (Equipment and Electronic Games)            442
Gas Ranges or Ovens                                   365
Microwave Ovens                                       362
Electric Coffee Makers or Teapots                     317
Nonmetal Cookware (Nonelectric)                       313
Refrigerators                                         278
Name: ProductCategoryPublicName, dtype: int64

In [106]:
#it appears most are just an incident with no real injury
data.SeverityTypePublicName.value_counts()

Incident, No Injury                               17916
No First Aid or Medical Attention Received         2672
First Aid Received by Non-Medical Professional     2177
Seen by Medical Professional                       1470
Unspecified                                        1330
Emergency Department Treatment Received            1090
Level of care not known                             715
Hospital Admission                                  456
No Incident, No Injury                              224
Missing                                             135
Death                                                93
Name: SeverityTypePublicName, dtype: int64

In [5]:
no_injuries = ['Incident, No Injury', 'Unspecified', 'Level of care not known',
               'No Incident, No Injury', 'No First Aid or Medical Attention Received']
damage = data.ix[~data['SeverityTypePublicName'].isin(no_injuries), :]
damage.SeverityTypePublicName.value_counts()

First Aid Received by Non-Medical Professional    2177
Seen by Medical Professional                      1470
Emergency Department Treatment Received           1090
Hospital Admission                                 456
Missing                                            135
Death                                               93
Name: SeverityTypePublicName, dtype: int64

In [124]:
# the item that causes the most damage would be footwear?
damage.ProductCategoryPublicName.value_counts()[0:9]

Footwear                                                    774
Computers (Equipment and Electronic Games)                  274
Diapers                                                     156
Electric Ranges or Ovens (Excl Counter-top Ovens)           134
Bicycles and Accessories, (Excl.mountain or All-terrain)    108
Baby Strollers                                              108
Electric Coffee Makers or Teapots                           100
Cribs                                                        94
Bassinets or Cradles                                         88
Name: ProductCategoryPublicName, dtype: int64

### Natural Language Processing

We'll need some to work parse out the incidents to see exactly what happened with footwear. For example, here are some of the complaints filed. So what we can do is to process the complaints and see what words are closely related to the injuries.

In [47]:
class parser(object):
    
    def __init__(self, report_list):
        self.report_list = report_list
        self.results = self.run_tokenizer()
     
    @staticmethod
    def format_sentence(sentence):
        if not isinstance(sentence, str):
            sentence = str(sentence)
        formatted = re.sub("[^a-zA-Z]"," ", sentence)
        formatted = formatted.lower()
        return formatted
    
    @staticmethod
    def tokenize_sentence(sentence):
        tokenized = nltk.word_tokenize(sentence)
        return tokenized
    
    def run_tokenizer(self):
        results = []
        for sentence in self.report_list:
            try:
                formatted = self.format_sentence(sentence)
                tokenized = self.tokenize_sentence(formatted)
                results.append(tokenized)
            except:
                continue
        return results
        
reports = [report for report in damage.ix[damage.ProductCategoryPublicName=='Footwear', 'IncidentDescription']]
parser = parser(reports)

In [ ]:
class modeler(object):
    
    def __init__(self, sentences, num_features=300,
                 min_word_count=40, num_workers=4, context=10,
                 downsampling=1e-3):
        self.sentences = sentences
        self.num_features = num_features                  
        self.min_word_count = num_features                      
        self.num_workers = num_workers
        self.context = context      
        self.downsampling = downsampling
        
    def train(self, only_once=True):
        model = word2vec.Word2Vec(self.sentences, workers=self.num_workers,
            size=self.num_features, min_count=self.min_word_count,
            window=self.context, sample=self.downsampling)
        if only_once:
            model.init_sims(replace=True)
        return model


# Appendix 

## Crosstab of Items and Injury

In [122]:
pd.crosstab(damage.ProductCategoryPublicName, damage.SeverityTypePublicName)

SeverityTypePublicName,Death,Emergency Department Treatment Received,First Aid Received by Non-Medical Professional,Hospital Admission,Missing,No First Aid or Medical Attention Received,Seen by Medical Professional
ProductCategoryPublicName,,,,,,,
< 5 Poisonings - No Other Code,0,1,0,0,0,0,0
Above-ground Swimming Pools (Excl Portable Pools),0,0,1,1,0,0,0
Adhesives,0,0,2,0,0,0,2
Aerosol Containers,0,1,1,0,0,2,1
Air Compressors (Separate),0,0,0,0,1,4,1
Air Conditioners,0,0,4,0,0,2,2
Air Purifiers,0,0,1,0,0,1,2
All Terrain Vehicles (# of Wheels Unspecified/off Road),4,3,0,0,0,0,0
All Terrain Vehicles (Four Wheels/off Road Only),12,3,6,6,0,5,1
